In [ ]:
import zipfile
import os
import pandas as pd
import numpy as np
import albumentations as A
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications import VGG16
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

ModuleNotFoundError: No module named 'albumentations'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp /content/drive/My\ Drive/COMP90086_2024_Project_train\(1\).zip /content/train.zip
!cp /content/drive/My\ Drive/COMP90086_2024_Project_test.zip /content/test.zip

cp: cannot stat '/content/drive/My Drive/COMP90086_2024_Project_train(1).zip': No such file or directory
cp: cannot stat '/content/drive/My Drive/COMP90086_2024_Project_test.zip': No such file or directory


In [ ]:
!unzip /content/train.zip -d /content/
!unzip /content/test.zip -d /content/

unzip:  cannot find or open /content/train.zip, /content/train.zip.zip or /content/train.zip.ZIP.
unzip:  cannot find or open /content/test.zip, /content/test.zip.zip or /content/test.zip.ZIP.


In [ ]:
augmentation = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.Rotate(limit=30, p=0.5),
    A.Normalize(mean=0.0, std=1.0),
])

In [ ]:
train_folder = '/content/train'
train_images = []
test_folder = '/content/test'
test_images = []

for image_file in os.listdir(train_folder):
      img = load_img(os.path.join(train_folder, image_file))
      img_array = img_to_array(img)
      train_images.append(img_array)

for image_file in os.listdir(test_folder):
      img = load_img(os.path.join(test_folder, image_file))
      img_array = img_to_array(img)
      test_images.append(img_array)


In [ ]:
train_images = np.array(train_images)
test_images = np.array(test_images)

In [ ]:
train_metadata = pd.read_csv('train.csv')

In [ ]:
X_train_metadata = train_metadata.drop('stable_height', axis=1).values
y_train = train_metadata['stable_height'].values

In [ ]:
X_train_images, X_val_images, X_train_tabular, X_val_tabular, y_train, y_val = train_test_split(
    train_images, X_train_metadata, y_train, test_size=0.2, random_state=42
)

In [ ]:
for img in X_train_images:
  augmented = augmentation(image=img)
  img = augmented['image']

In [ ]:
tabular_input = layers.Input(shape=(X_train_metadata.shape[1],))
output1 = layers.Dense(64, activation='relu')(tabular_input)

image_input = layers.Input(shape=(train_images.shape[1], train_images.shape[2], 3))
vgg_model = VGG16(weights='imagenet', include_top=False, input_tensor=image_input)
output2 = layers.Flatten()(vgg_model.output)
output2 = layers.Dense(64, activation='relu')(output2)

combined = layers.concatenate([output1, output2])
output3 = layers.Dense(64, activation='relu')(combined)
output = layers.Dense(1, activation='linear')(output3)

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [ ]:
model = keras.Model(inputs=[image_input, tabular_input], outputs=output)
model.compile(optimizer='adam', loss=keras.losses.MeanSquaredError(), metrics=['accuracy'])

In [ ]:
model_history = model.fit([X_train_images, X_train_tabular], y_train,
                          validation_data=([X_val_images, X_val_tabular], y_val),
                          epochs=20, batch_size=100)